In [2]:
#from sum_ import sum_
from clean_func import EV_info, clean_pmt_type, GetCoordinateshitPMT
from analysis_simulation import Analysis_simulation
import uproot
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
import seaborn as sn
from tqdm import tqdm

import numpy as np
from numpy import array, where, shape, reshape

import pandas as pd

import uproot

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
def magnitude(vector): 
    return np.sqrt(sum(pow(element, 2) for element in vector))

In [4]:
file = uproot.open("/sno/py_out1.root")

In [3]:
IDs = [1,2,50,31,23,11]
#IDs = np.arange(0,51)
Analysis_simulation(file, IDs)

,eventID,hitpmt ID,hit type,mc radius,reconst radius,reconst error
0,1,"[8967, 539, 998, 1015, 1496, 987, 931, 493, 92...","[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, ...",4424.431649,4393.412025,189.052592
1,2,"[9695, 9039, 9586, 9156, 9673, 9072, 9675, 956...","[2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, ...",3987.383832,4029.665232,98.109757
2,50,"[3101, 5526, 4806, 5302, 5200, 6218, 6237, 517...","[2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, ...",2272.642333,2406.404738,229.281677
3,31,"[8080, 9385, 4967, 4972, 5543, 4977, 4810, 469...","[2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, ...",2891.501632,2908.134733,130.861423
4,23,"[4604, 1801, 1858, 1403, 2246, 2245, 2324, 177...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, ...",5067.835612,4979.220167,93.709500
5,11,"[2579, 8519, 8484, 8966, 8511, 8507, 8486, 897...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",5394.239748,5421.818817,48.522873


In [5]:
type(IDs)

list

In [7]:
from typing import Union

In [8]:
r, _ = GetCoordinateshitPMT(file, [1,7], 15)

In [9]:
len(r)

4735

In [10]:
r

array([[ 5467.54,    63.44, -6406.27],
       [ 5615.16,   266.8 , -6251.09],
       [ 5762.73,   469.91, -6095.92],
       ...,
       [-4287.58,  6152.61,  3796.11],
       [-6600.28,  3114.85,  4197.48],
       [ 2545.4 , -7078.81,  3737.76]])

In [23]:
pos = np.array([[1,2],[3,4],[5,6]])
evID = 0
pos[0]

array([1, 2])

In [30]:
e = np.array([], dtype = np.int32)

for i in [0,2]:
    e = np.append(e,pos[i])
    
e

array([1, 2, 5, 6])

In [7]:
# Simulation Info
data1 = file['T;7']
#Extract Simulation Data
evtid = np.array(data1['evtid'])
mc_position = np.array(data1['mc_position'])
position = np.array(data1['position'])
hit_pmtid = np.array(data1['hit_pmtid'])
hit_pmttime = np.array(data1['hit_pmttime'])
hit_residual = np.array(data1['hit_residual'])
hit_type = np.array(data1['hit_type'])

#PMT Information
pmt_info = file['pmt;1']
#Extract PMT info.
pmt_id = array(pmt_info['pmt_id'])
pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
pmt_type = array(pmt_info['pmt_type'])


mc_rad_ID = np.array([])							# mc event radio
recons_rad_ID = np.array([])    					# reconstructed radio
reconstructed_error_ID = np.array([])				# reconstruction error (magnitude of diference os position vectors)
hit_pmtid_ID = []		#ID of PMTs giving hits


evID = [11, 50]

for i_id in evID:

    #When simulation data includes many simulations the IDs are repeated, then execute the following to separate subsets:
    condition_1 = (evtid == i_id)
    index = where(condition_1)[0]               
    break_data = np.array([index[0]], dtype = np.int64)    #indices where data Breaks - Indices where subsets start
    # extract the index where data breaks
    for i in range(len(index)-1):
        if index[i+1] != index[i]+1:
            break_data = np.append(break_data,index[i+1])

    break_data = np.append(break_data,index[-1]+1)

    subset = [0,1]  #first subset

    init =  break_data[subset[0]]
    final_ = break_data[subset[1]]

    #Extract new data of subset: 
    sub_evtid = evtid[init:final_]
    sub_mc_position = mc_position[init:final_]
    sub_position = position[init:final_]
    #print(sub_position)
    sub_hit_pmtid = hit_pmtid[init:final_]
    sub_hit_pmttime = hit_pmttime[init:final_]
    sub_hit_residual = hit_residual[init:final_]
    sub_hit_type = hit_type[init:final_]

    #Retaking data of eventID desired in subset
    condition_2 = (sub_evtid == i_id)
    pmtid_ev = np.extract(condition_2, sub_hit_pmtid) 			#Indices for which elements are True in eventID then extract the hit_pmtID
    time_res_ev = np.extract(condition_2, sub_hit_residual)
    pmttime_ev = np.extract(condition_2, sub_hit_pmttime)
    pmt_type_ev = np.extract(condition_2, sub_hit_type)

    #Positions and radi
    mc_position_ev = []
    recons_position_ev = []
    mc_radius_ev = np.array([])
    recons_radius_ev = np.array([])
    for i in where(condition_2)[0]:
        mc_position_ev.append(sub_mc_position[i])
        recons_position_ev.append(sub_position[i])
    mc_position_ev = np.array(mc_position_ev)
    recons_position_ev = np.array(recons_position_ev)
   # print(recons_position_ev)

    for i in range(shape(mc_position_ev)[0]):
        MC_radius = magnitude(mc_position_ev[i])
        RECONS_radius = magnitude(recons_position_ev[i])
        mc_radius_ev = np.append(mc_radius_ev, RECONS_radius)
        recons_radius_ev = np.append(recons_radius_ev, RECONS_radius)

    #APPEND
    recons_rad_ID = np.append(recons_rad_ID, recons_radius_ev[0])
    mc_rad_ID = np.append(mc_rad_ID, mc_radius_ev[0])

    #Calculate error of reconstruction
    mc_xyz = mc_position_ev[0]
    recons_xyz = recons_position_ev[0]
    dif_v = mc_xyz - recons_xyz											#vector diference
    error = magnitude(dif_v)											#recontruction error - a simple calculation
    #APPEND
    reconstructed_error_ID = np.append(reconstructed_error_ID, error)

    #PMT type selection
    valid_type = 1
    valid_pmt_id = np.array([],dtype = np.int64)	#valid PMTid

    valid_info_pmt_i = where(pmt_type == valid_type)[0]

    for i in valid_info_pmt_i:
        valid_pmt_id = np.append(valid_pmt_id, pmt_id[i])


    # Buscar información valida dentro del eventos:

    #indices de info. valida
    valid_info_ev_i = where(np.in1d(pmtid_ev, valid_pmt_id))

    #información que deseo observar
    time_res_ev_valid = np.array([])
    pmt_id_ev_valid = np.array([], dtype = np.int64)

    for j in valid_info_ev_i:
        time_res_ev_valid = np.append(time_res_ev_valid, time_res_ev[j])
        pmt_id_ev_valid = np.append(pmt_id_ev_valid, pmtid_ev[j])

    #APPEND
    hit_pmtid_ID.append(pmt_id_ev_valid)

    #Save useful plots
#     hist(time_res_ev_valid, 100, 'time_res_ev', title = 'ID=' +str(evID))
#     hist(mc_radius_ev, 100, 'mc_radius_ev',  'data quantity', title = 'ID = ' + str(evID))

#Construct DataFrame
data = {'eventID': evID,
        'hitpmt ID': hit_pmtid_ID,
        'mc radius': mc_rad_ID,
        'reconst radius': recons_rad_ID,
        'reconst error': reconstructed_error_ID,
        }
DataFrame_analysis = pd.DataFrame(data)

In [30]:
shape(hit_pmtid_ID)

(1559,)

In [22]:
mc_rad_ID

array([5421.81881698, 2406.40473757])

In [24]:
recons_rad_ID

array([5421.81881698, 2406.40473757])

In [68]:
hit_pmtid_ID[0]

array([2579, 8519, 8484, 8966, 8511, 8507, 8486, 8976, 8964, 7949, 8510,
       8661, 7908, 8118, 7936, 7961, 8523, 8949, 8967, 8970, 9141, 8535,
       8500, 7909, 8459, 8527, 8481, 8489, 7899, 8948, 8666, 9122, 8466,
       8961, 9162, 8498, 7990, 8975, 8457, 8485, 7935, 7965, 7954, 9155,
       7864, 7973, 9167, 7916, 8465, 8471, 9118, 8451, 7952, 7938, 8516,
       8514, 7918, 8986, 8476, 9116, 9126, 8143, 9662, 7869, 8517, 8540,
       7946, 8938, 9684, 8983, 8483, 9109, 7978, 8549, 7838, 7902, 8496,
       8462, 8971, 8534, 7867, 8508, 8532, 9172, 7957, 9135, 8530, 8443,
       7834, 7880, 9149, 7898, 8935, 9120, 9628, 8654, 7839, 8981, 7969,
       1677, 7846, 8972, 7962, 7964, 8942, 7404, 7982, 8005, 1690, 8643,
       7981, 9689, 9642, 9171, 8550, 7986, 9180, 7897, 8453, 8965, 8974,
       9669, 7991, 8487, 9089, 7877, 9138, 8439, 9087,  229, 8644,  726,
       9695, 9173, 8555, 8482, 8973, 7390, 8124, 7925, 1700, 7429, 9646,
       9159, 9097, 9613,  724, 8670, 8134, 7882, 89

In [58]:
shape(hit_pmtid_ID)

(1, 1559)

In [8]:
DataFrame_analysis

,eventID,hitpmt ID,mc radius,reconst radius,reconst error
0,11,"[2579, 8519, 8484, 8966, 8511, 8507, 8486, 897...",5421.818817,5421.818817,48.522873
1,50,"[3101, 5526, 4806, 5302, 5200, 6218, 6237, 517...",2406.404738,2406.404738,229.281677


## Transformation of PMT coordinates

$\phi = 0 $ (zenith) in the $xy$ plane

In [5]:
pmt_info = file['pmt;1']
pmt_info.keys()

['pmt_id', 'pmt_pos_xyz', 'pmt_pos_sph', 'pmt_type']

In [6]:
pmt_id = array(pmt_info['pmt_id'])
pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
pmt_type = array(pmt_info['pmt_type'])

In [21]:
pmt_pos_sph

array([[ 2.18627604e+00, -2.35619449e+00,  1.73203349e+05],
       [ 2.43505602e+00,  1.16025037e-02,  8.42248856e+03],
       [ 2.40917419e+00,  4.74785216e-02,  8.40698104e+03],
       ...,
       [ 6.54226614e-01, -1.48191511e+00,  9.20079013e+03],
       [ 7.32698456e-01, -8.70981952e-01,  9.20131295e+03],
       [ 3.22890171e-01, -8.01377433e-01,  9.20467647e+03]])

In [22]:
pmt_pos_sph[:,0] += np.pi/2

In [23]:
pmt_pos_sph

array([[ 3.75707236e+00, -2.35619449e+00,  1.73203349e+05],
       [ 4.00585235e+00,  1.16025037e-02,  8.42248856e+03],
       [ 3.97997052e+00,  4.74785216e-02,  8.40698104e+03],
       ...,
       [ 2.22502294e+00, -1.48191511e+00,  9.20079013e+03],
       [ 2.30349478e+00, -8.70981952e-01,  9.20131295e+03],
       [ 1.89368650e+00, -8.01377433e-01,  9.20467647e+03]])